In [ ]:
import numpy as np
import os
from tensorflow.keras.utils import to_categorical

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
actions = [
    'flower',
    'crown',
    'heart_beat',
    'firework',
    'bear',
    'cat',
    'son_celebration',
    'heart_on_the_cheek',
    'gun',
    'pipe',
    'tiger',
    'landmarks'
]

data = np.concatenate([
    np.load('new_dataset/seq_flower_1750130471.npy'),
    np.load('new_dataset/seq_crown_1750130505.npy'),
    np.load('new_dataset/seq_heart_beat_1750139892.npy'),
    np.load('new_dataset/seq_firework_1750140689.npy'),
    np.load('new_dataset/seq_bear_1750140330.npy'),
    np.load('new_dataset/seq_cat_1750140596.npy'),
    np.load('new_dataset/seq_son_celebration_1750139350.npy'),
    np.load('new_dataset/seq_Hear_on_the_cheek_1750139384.npy'),
    np.load('new_dataset/seq_gun_1750139891.npy'),
    np.load('new_dataset/seq_pipe_1750140043.npy'),
    np.load('new_dataset/seq_tiger_1750140076.npy'),
    np.load('new_dataset/seq_landmarks_1750140109.npy'),
], axis=0)

data.shape

In [ ]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

In [ ]:
# float로 되어 있다면 반드시 int로 변환
labels = np.array(labels).astype(int)

# num_classes 정확히 계산
num_classes = int(np.max(labels)) + 1

# 원-핫 인코딩
y_data = to_categorical(labels, num_classes=num_classes)
print(y_data.shape)

In [ ]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    LSTM(64, input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

optimizer = Adam(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

EPOCHS = 30

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

테스트

In [ ]:
# 시퀀스 30
test_data = np.concatenate([
    np.load('test_dataset/seq_flower_1750301079.npy'),
    np.load('test_dataset/seq_crown_1750301143.npy'),
    np.load('test_dataset/seq_heartbeat_1750301206.npy'),
    np.load('test_dataset/seq_firework_1750301269.npy'),
    np.load('test_dataset/seq_bear_1750301332.npy'),
    np.load('test_dataset/seq_cat_1750301396.npy'),
    np.load('test_dataset/seq_son_celebration_1750301585.npy'),
    np.load('test_dataset/seq_heart_on_the_cheek_1750301522.npy'),
    np.load('test_dataset/seq_gun_1750301459.npy'),
    np.load('test_dataset/seq_pipe_1750301711.npy'),
    np.load('test_dataset/seq_tiger_1750301648.npy'),
    np.load('test_dataset/seq_landmarks_1750301774.npy'),
], axis=0)

# 시퀀스 60
#test_data = np.concatenate([
#    np.load('test_dataset60/seq_flower_1750299062.npy'),
#    np.load('test_dataset60/seq_crown_1750299126.npy'),
#    np.load('test_dataset60/seq_heartbeat_1750299189.npy'),
#    np.load('test_dataset60/seq_firework_1750299274.npy'),
#    np.load('test_dataset60/seq_bear_1750299338.npy'),
#    np.load('test_dataset60/seq_cat_1750299401.npy'),
#    np.load('test_dataset60/seq_son_celebration_1750299613.npy'),
#    np.load('test_dataset60/seq_heart_on_the_cheek_1750299550.npy'),
#    np.load('test_dataset60/seq_gun_1750299486.npy'),
#    np.load('test_dataset60/seq_pipe_1750299766.npy'),
#    np.load('test_dataset60/seq_tiger_1750299702.npy'),
#    np.load('test_dataset60/seq_landmarks_1750299829.npy'),
#], axis=0)

test_data.shape

In [ ]:
x_test = test_data[:, :, :-1]
test_labels = test_data[:, 0, -1]

print(x_test.shape)
print(test_labels.shape)

In [ ]:
# float로 되어 있다면 반드시 int로 변환
test_labels = np.array(test_labels).astype(int)

# num_classes 정확히 계산
num_classes2 = int(np.max(test_labels)) + 1

# 원-핫 인코딩
y_test = to_categorical(test_labels, num_classes=num_classes2)
print(y_test.shape)

In [ ]:
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model_new_hyperparameter.h5')

y_pred = model.predict(x_test)

multilabel_confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 예측
y_pred = model.predict(x_test)
y_true = np.argmax(y_test, axis=1)
y_pred_label = np.argmax(y_pred, axis=1)

# Confusion matrix (전체 클래스 기준)
cm = confusion_matrix(y_true, y_pred_label)

# 클래스 이름
class_names = actions  # 예: ['flower', 'crown', ..., 'landmarks']

# 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('12x12 Confusion Matrix')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:

from sklearn.metrics import classification_report

y_pred_class = np.argmax(y_pred, axis=1)
print("\nClassification Report:")
print(classification_report(y_true, y_pred_class, target_names=actions))
